In [1]:
## importing the libraries
import pandas as pd
import numpy as np
import os
import ast
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder

## setting display to show maximum 500 rows and columns
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

## ignoring the warnings
import warnings
warnings.simplefilter("ignore", UserWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None

In [3]:
## reading all files
all_recommendations = pd.read_csv(r"..\data\processed_data\all_recommendations_0.8_top5.csv")
RAF_Age_gender = pd.read_excel(r"..\data\RAF Factors data\V28_HCC_Factors_unpivoted.xlsx",sheet_name="Age and gender_p")
RAF_Medicaid_OrgnlDsbl = pd.read_excel(r"..\data\RAF Factors data\V28_HCC_Factors_unpivoted.xlsx",sheet_name="Medicaid and OriginallyDisabl_p")
RAF_Disease_Coefficients = pd.read_excel(r"..\data\RAF Factors data\V28_HCC_Factors_unpivoted.xlsx",sheet_name="Disease Coefficients_p")
RAF_Disease_intraction = pd.read_excel(r"..\data\RAF Factors data\V28_HCC_Factors_unpivoted.xlsx",sheet_name="Disease intraction_p")
RAF_Disabled_Disease_Int = pd.read_excel(r"..\data\RAF Factors data\V28_HCC_Factors_unpivoted.xlsx",sheet_name="Disabled Disease Interactions_p")
RAF_Payment_HCC_Counts = pd.read_excel(r"..\data\RAF Factors data\V28_HCC_Factors_unpivoted.xlsx",sheet_name="Payment HCC Counts_p")
hirarcial_conditions = pd.read_excel(r"..\data\RAF Factors data\V28_hirarcial_conditions.xlsx")

In [7]:
## converting strings into lists and sets
def string_to_list(s):
    try:
        return ast.literal_eval(s)
    except (ValueError, SyntaxError):
        return []
    
# Apply the string_to_list function to each element of the dataframe
columns = ["Target_HCC","Simialr_HCC","common_HCC","individual_recommended_HCC",
           "combined_recommedned_HCC","combined_recommedned_HCC_trimmed"]
all_recommendations[columns] = all_recommendations[columns].applymap(string_to_list)
hirarcial_conditions["drop_hcc"] = hirarcial_conditions["drop_hcc"].apply(string_to_list)

In [80]:
## taking only top 2 from trimmed recommendations
top_positive_recommednations = 2
all_recommendations["combined_recommedned_HCC_trimmed"] = [x[:top_positive_recommednations]
                                                           for x in all_recommendations["combined_recommedned_HCC_trimmed"]]

In [81]:
# age conversion of RAF AGE Gender data
RAF_Age_gender["age_from"] = RAF_Age_gender["AGE"].apply(lambda x: x.split(' ')[0])
RAF_Age_gender["age_to"] = RAF_Age_gender["AGE"].apply(lambda x: x.split(' ')[2]).replace({"or" : 1000})
RAF_Age_gender.drop(columns=["AGE"],inplace=True)

In [82]:
## transforming RAF_Disabled_Disease_Int data
RAF_Disabled_Disease_Int["DISABLED_HF"] = np.where(RAF_Disabled_Disease_Int["Disabled/Disease Interactions"]=="DISABLED_HF",1,0)
RAF_Disabled_Disease_Int["DISABLED_ULCER_"] = np.where(RAF_Disabled_Disease_Int["Disabled/Disease Interactions"]=="DISABLED_ULCER_",1,0)
RAF_Disabled_Disease_Int["DISABLED_CANCER"] = np.where(RAF_Disabled_Disease_Int["Disabled/Disease Interactions"]=="DISABLED_CANCER",1,0)
RAF_Disabled_Disease_Int["DISABLED_NEURO_"] = np.where(RAF_Disabled_Disease_Int["Disabled/Disease Interactions"]=="DISABLED_NEURO_",1,0)
RAF_Disabled_Disease_Int["DISABLED_CHR_LUNG"] = np.where(RAF_Disabled_Disease_Int["Disabled/Disease Interactions"]=="DISABLED_CHR_LUNG",1,0)
RAF_Disabled_Disease_Int.drop(columns=['Disabled/Disease Interactions'],inplace=True)

In [83]:
## transforming RAF_Disease_intraction data
RAF_Disease_intraction["DIABETES_HF"] = np.where(RAF_Disease_intraction["Disease_intraction"]=="DIABETES_HF",1,0)
RAF_Disease_intraction["HF_CHR_LUNG"] = np.where(RAF_Disease_intraction["Disease_intraction"]=="HF_CHR_LUNG",1,0)
RAF_Disease_intraction["HF_KIDNEY"] = np.where(RAF_Disease_intraction["Disease_intraction"]=="HF_KIDNEY",1,0)
RAF_Disease_intraction["CHR_LUNG_CARD_RESP_FAIL"] = np.where(RAF_Disease_intraction["Disease_intraction"]=="CHR_LUNG_CARD_RESP_FAIL",1,0)
RAF_Disease_intraction["HF_HCC238"] = np.where(RAF_Disease_intraction["Disease_intraction"]=="HF_HCC238",1,0)
RAF_Disease_intraction["gSubUseDisorder_gPsych_"] = np.where(RAF_Disease_intraction["Disease_intraction"]=="gSubUseDisorder_gPsych_",1,0)
RAF_Disease_intraction.drop(columns=['Disease_intraction'],inplace=True)

In [84]:
## transforming HCC codes and converting it into int
RAF_Disease_Coefficients['HCC_code'] = [x[3:] for x in RAF_Disease_Coefficients['HCC_code']]
RAF_Disease_Coefficients['HCC_code'] = RAF_Disease_Coefficients['HCC_code'].astype(int)

In [85]:
## transforming RAF_Payment_HCC_Counts data
RAF_Payment_HCC_Counts['Description Label'] = [int(x[0]) for x in RAF_Payment_HCC_Counts['Description Label'].str.split(' ')]

In [86]:
## transforming all_recommendations data
all_recommendations["Gender"] = np.where(all_recommendations["Gender"] == 1, 'Male', 'Female')
all_recommendations.rename(columns={'Benefits':'benefits'},inplace=True)
all_recommendations["Aged_disabled"] = np.where(all_recommendations["Aged"]== 1, 'Aged', 'Disabled')
all_recommendations["OriginallyDisabled_Medicaid"] = np.where(all_recommendations["Disability_condn"] == 'Originally_disabled ', 'Originally Disabled', 'Medicaid')
all_recommendations.rename(columns={'Disease_intraction_DIABETES_HF':'DIABETES_HF',
                                    'Disease_intraction_HF_CHR_LUNG': 'HF_CHR_LUNG',
                                    'Disease_intraction_HF_KIDNEY': 'HF_KIDNEY',
                                    'Disease_intraction_CHR_LUNG_CARD_RESP_FAIL': 'CHR_LUNG_CARD_RESP_FAIL',
                                    'Disease_intraction_HF_HCC238': 'HF_HCC238',
                                    'Disease_intraction_gSubUseDisorder_gPsych_':'gSubUseDisorder_gPsych_'},inplace=True)

all_recommendations['Target_HCC'] = [list(x) for x in all_recommendations['Target_HCC']]
all_recommendations["HCC_after_recommendations"] = all_recommendations['Target_HCC'] + all_recommendations['combined_recommedned_HCC_trimmed']
all_recommendations['count_HCC_before_recommendations'] = [ len(x) for x in all_recommendations['Target_HCC']]
all_recommendations['count_HCC_after_recommendations'] = [ len(x) for x in all_recommendations['HCC_after_recommendations']]


In [87]:
## creaing list of standard disease code list

DIABETES = [35,36,37,38]
HF = [221,222,223,224,225,226] 
LUNG_DISORDER = [276,277,278,279,280]
KIDNEY = [326, 327, 328, 329]
Cardio_Respiratory = [211,212,213]
Heart_Arrhythmias = [238]
SUBSTANCE_Use_Disorder = [137, 138, 139]
SKIN_ULCER = [379, 380, 381, 382]
CANCER = [17, 18, 19, 20, 21, 22, 23]
neuological = [180,181,182,183,184,185,186,187,188,189,190,191,192,
              195,196,198,199]
psychiatric = [151,152,153,154,155]
substance_use = [135,136,137,138,139]

In [88]:
## finding disease intractions after recommendation

# Define a function to check if all elements of a list are present in another list
def check_in_list(lst1, lst2):
    return any([elem in lst2 for elem in lst1])

# Apply the function to each row of the dataframe
all_recommendations['DIABETES_HF'] = all_recommendations['HCC_after_recommendations'].apply(lambda x: 1 if check_in_list(DIABETES, x) and check_in_list(HF, x) else 0)
all_recommendations['HF_CHR_LUNG'] = all_recommendations['HCC_after_recommendations'].apply(lambda x: 1 if check_in_list(LUNG_DISORDER, x) and check_in_list(HF, x) else 0)
all_recommendations['HF_KIDNEY'] = all_recommendations['HCC_after_recommendations'].apply(lambda x: 1 if check_in_list(KIDNEY, x) and check_in_list(HF, x) else 0)
all_recommendations['CHR_LUNG_CARD_RESP_FAIL'] = all_recommendations['HCC_after_recommendations'].apply(lambda x: 1 if check_in_list(LUNG_DISORDER, x) and check_in_list(Cardio_Respiratory, x) else 0)
all_recommendations['HF_HCC238'] = all_recommendations['HCC_after_recommendations'].apply(lambda x: 1 if check_in_list(Heart_Arrhythmias, x) and check_in_list(HF, x) else 0)
all_recommendations['gSubUseDisorder_gPsych_'] = all_recommendations['HCC_after_recommendations'].apply(lambda x: 1 if check_in_list(substance_use, x) and check_in_list(psychiatric, x) else 0)

In [89]:
## finding disease intraction after recommendation

# Define a function to check if all elements of a list are present in another list
def check_in_list(lst1, lst2):
    return any([elem in lst2 for elem in lst1])

all_recommendations_disable = all_recommendations[~(all_recommendations["Disability_condn"] =='Non_disabled ')]
all_recommendations_Non_disable = all_recommendations[(all_recommendations["Disability_condn"] =='Non_disabled ')]

# Apply the function to each row of the dataframe
all_recommendations_disable['DISABLED_HF'] = all_recommendations_disable['HCC_after_recommendations'].apply(lambda x: 1 if check_in_list(HF, x) else 0)
all_recommendations_disable['DISABLED_ULCER_'] = all_recommendations_disable['HCC_after_recommendations'].apply(lambda x: 1 if check_in_list(SKIN_ULCER, x) else 0)
all_recommendations_disable['DISABLED_CANCER'] = all_recommendations_disable['HCC_after_recommendations'].apply(lambda x: 1 if check_in_list(CANCER, x) else 0)
all_recommendations_disable['DISABLED_NEURO_'] = all_recommendations_disable['HCC_after_recommendations'].apply(lambda x: 1 if check_in_list(neuological, x) else 0)
all_recommendations_disable['DISABLED_CHR_LUNG'] = all_recommendations_disable['HCC_after_recommendations'].apply(lambda x: 1 if check_in_list(LUNG_DISORDER, x) else 0)

all_recommendations_Non_disable['DISABLED_HF'] = 0
all_recommendations_Non_disable['DISABLED_ULCER_'] = 0
all_recommendations_Non_disable['DISABLED_CANCER'] = 0
all_recommendations_Non_disable['DISABLED_NEURO_'] = 0
all_recommendations_Non_disable['DISABLED_CHR_LUNG'] = 0

all_recommendations = all_recommendations_disable.append(
                            all_recommendations_Non_disable[all_recommendations_disable.columns.to_list()]).sort_index()

In [90]:
%%time
## dropping HCC's through hirarical condition
def drop_hcc(hcc_list):
    for hcc in hcc_list:
        if hcc in hirarcial_conditions['HCC'].values:
            drop_list = hirarcial_conditions.loc[hirarcial_conditions['HCC'] == hcc, 'drop_hcc'].values[0]
            hcc_list = [h for h in hcc_list if h not in drop_list]
    return hcc_list

all_recommendations['HCC_after_recommendations'] = all_recommendations['HCC_after_recommendations'].apply(drop_hcc)
all_recommendations['Target_HCC'] = all_recommendations['Target_HCC'].apply(drop_hcc)

CPU times: total: 5min 21s
Wall time: 5min 24s


In [91]:
all_recommendations_c = all_recommendations[all_recommendations['community_institutional']=='Community']
all_recommendations_I = all_recommendations[all_recommendations['community_institutional']=='Institutional']

In [92]:
## finding Risk factor due AGE and Gender
all_recommendations_c = pd.merge(
    all_recommendations_c, RAF_Age_gender,
    on=['Gender', 'community_institutional', 'benefits', 'Aged_disabled'],
    how = 'left').rename(columns={"HCC_factor":"RAF_Age_gender"})

all_recommendations_I = pd.merge(
    all_recommendations_I, RAF_Age_gender.drop(columns=['benefits', 'Aged_disabled']),
    on=['Gender', 'community_institutional'],
    how = 'left').rename(columns={"HCC_factor":"RAF_Age_gender"})

all_recommendations_c[["age_from","age_to"]] = all_recommendations_c[["age_from","age_to"]].fillna(0).astype(int)
all_recommendations_I[["age_from","age_to"]] = all_recommendations_I[["age_from","age_to"]].fillna(0).astype(int)

# filter rows where age is between age_from and age_to
all_recommendations_c = all_recommendations_c.loc[(all_recommendations_c['Age'] >= all_recommendations_c['age_from']) & (all_recommendations_c['Age'] <= all_recommendations_c['age_to'])].reset_index(drop=True)
all_recommendations_I = all_recommendations_I.loc[(all_recommendations_I['Age'] >= all_recommendations_I['age_from']) & (all_recommendations_I['Age'] <= all_recommendations_I['age_to'])].reset_index(drop=True)

In [93]:
## finding Risk factor due to medicaid or originally disabled conditions
all_recommendations_c = pd.merge(all_recommendations_c,
                               RAF_Medicaid_OrgnlDsbl,
                               on=['OriginallyDisabled_Medicaid','Gender','community_institutional',
                                   'benefits','Aged_disabled'],
                               how='left'
                              ).rename(columns={"HCC_factor":"RAF_Medicaid_OrgnlDsbl"})

all_recommendations_I = pd.merge(all_recommendations_I,
                               RAF_Medicaid_OrgnlDsbl.drop(columns=['benefits','Aged_disabled']),
                               on=['OriginallyDisabled_Medicaid','Gender','community_institutional'],
                               how='left'
                              ).rename(columns={"HCC_factor":"RAF_Medicaid_OrgnlDsbl"})

In [94]:
## finding Risk factor due to HCC's before recommendations

all_recommendations_exploded_c = all_recommendations_c[['Target_Beneficiary','Target_HCC','community_institutional','benefits','Aged_disabled']].drop_duplicates(subset=["Target_Beneficiary"]).explode('Target_HCC')
all_recommendations_exploded_I = all_recommendations_I[['Target_Beneficiary','Target_HCC','community_institutional','benefits','Aged_disabled']].drop_duplicates(subset=["Target_Beneficiary"]).explode('Target_HCC')


all_recommendations_exploded_c = pd.merge(all_recommendations_exploded_c,
                               RAF_Disease_Coefficients,
                               left_on=['Target_HCC','community_institutional','benefits','Aged_disabled'],
                               right_on = ['HCC_code','community_institutional','Benefits','Aged_disabled'],
                               how='left'
                              ).rename(columns={"HCC_factor":"RAF_Disease_Coefficients_before_recommendations"})

all_recommendations_exploded_I = pd.merge(all_recommendations_exploded_I,
                               RAF_Disease_Coefficients.drop(columns=['Benefits','Aged_disabled']),
                               left_on=['Target_HCC','community_institutional'],
                               right_on = ['HCC_code','community_institutional'],
                               how='left'
                              ).rename(columns={"HCC_factor":"RAF_Disease_Coefficients_before_recommendations"})

df_HCC_Disease_Coefficients_sum_before_c = all_recommendations_exploded_c[['Target_Beneficiary','RAF_Disease_Coefficients_before_recommendations']].groupby("Target_Beneficiary").sum().reset_index()
df_HCC_Disease_Coefficients_sum_before_I = all_recommendations_exploded_I[['Target_Beneficiary','RAF_Disease_Coefficients_before_recommendations']].groupby("Target_Beneficiary").sum().reset_index()

all_recommendations_c = pd.merge(all_recommendations_c,df_HCC_Disease_Coefficients_sum_before_c, on='Target_Beneficiary',how='left')
all_recommendations_I = pd.merge(all_recommendations_I,df_HCC_Disease_Coefficients_sum_before_I, on='Target_Beneficiary',how='left')

In [95]:
## finding Risk factor due to HCC's after recommendations

all_recommendations_exploded_c = all_recommendations_c[['Target_Beneficiary','HCC_after_recommendations','community_institutional','benefits','Aged_disabled']].drop_duplicates(subset=["Target_Beneficiary"]).drop_duplicates(subset=["Target_Beneficiary"]).explode('HCC_after_recommendations')
all_recommendations_exploded_I = all_recommendations_I[['Target_Beneficiary','HCC_after_recommendations','community_institutional','benefits','Aged_disabled']].drop_duplicates(subset=["Target_Beneficiary"]).drop_duplicates(subset=["Target_Beneficiary"]).explode('HCC_after_recommendations')

all_recommendations_exploded_c = pd.merge(all_recommendations_exploded_c,
                               RAF_Disease_Coefficients,
                               left_on=['HCC_after_recommendations','community_institutional','benefits','Aged_disabled'],
                               right_on = ['HCC_code','community_institutional','Benefits','Aged_disabled'],
                               how='left'
                              ).rename(columns={"HCC_factor":"RAF_Disease_Coefficients_after_recommendations"})
all_recommendations_exploded_I = pd.merge(all_recommendations_exploded_I,
                               RAF_Disease_Coefficients.drop(columns=['Benefits','Aged_disabled']),
                               left_on=['HCC_after_recommendations','community_institutional'],
                               right_on = ['HCC_code','community_institutional'],
                               how='left'
                              ).rename(columns={"HCC_factor":"RAF_Disease_Coefficients_after_recommendations"})

df_HCC_Disease_Coefficients_sum_after_c = all_recommendations_exploded_c[['Target_Beneficiary','RAF_Disease_Coefficients_after_recommendations']].groupby("Target_Beneficiary").sum().reset_index()
df_HCC_Disease_Coefficients_sum_after_I = all_recommendations_exploded_I[['Target_Beneficiary','RAF_Disease_Coefficients_after_recommendations']].groupby("Target_Beneficiary").sum().reset_index()

all_recommendations_c = pd.merge(all_recommendations_c,df_HCC_Disease_Coefficients_sum_after_c, on='Target_Beneficiary',how='left')
all_recommendations_I = pd.merge(all_recommendations_I,df_HCC_Disease_Coefficients_sum_after_I, on='Target_Beneficiary',how='left')

In [96]:
## finding Risk factor due to disease intraction conditions
all_recommendations_c = pd.merge(all_recommendations_c,
                               RAF_Disease_intraction,
                               left_on=['community_institutional', 'benefits', 'Aged_disabled',
                                   'DIABETES_HF', 'HF_CHR_LUNG', 'HF_KIDNEY', 'CHR_LUNG_CARD_RESP_FAIL',
                                   'HF_HCC238','gSubUseDisorder_gPsych_'],
                               right_on=['community_institutional', 'benefits', 'Aged_disabled',
                                   'DIABETES_HF', 'HF_CHR_LUNG', 'HF_KIDNEY', 'CHR_LUNG_CARD_RESP_FAIL',
                                   'HF_HCC238','gSubUseDisorder_gPsych_'],
                               how='left'
                              ).rename(columns={"HCC_factor":"RAF_Disease_intraction"})

all_recommendations_I = pd.merge(all_recommendations_I,
                               RAF_Disease_intraction.drop(columns=['benefits', 'Aged_disabled']),
                               left_on=['community_institutional',
                                   'DIABETES_HF', 'HF_CHR_LUNG', 'HF_KIDNEY', 'CHR_LUNG_CARD_RESP_FAIL',
                                   'HF_HCC238','gSubUseDisorder_gPsych_'],
                               right_on=['community_institutional',
                                   'DIABETES_HF', 'HF_CHR_LUNG', 'HF_KIDNEY', 'CHR_LUNG_CARD_RESP_FAIL',
                                   'HF_HCC238','gSubUseDisorder_gPsych_'],
                               how='left'
                              ).rename(columns={"HCC_factor":"RAF_Disease_intraction"})

all_recommendations_c['RAF_Disease_intraction'] = all_recommendations_c['RAF_Disease_intraction'].fillna(0)
all_recommendations_I['RAF_Disease_intraction'] = all_recommendations_I['RAF_Disease_intraction'].fillna(0)

In [97]:
## finding Risk factor due to disability and disease intraction conditions
all_recommendations_c['RAF_Disabled_Disease_Int'] = 0

all_recommendations_I = pd.merge(all_recommendations_I,
                               RAF_Disabled_Disease_Int[RAF_Disabled_Disease_Int['DISABLED_HF']==1][['community_institutional', 'HCC_factor', 'DISABLED_HF']],
                               on=['community_institutional','DISABLED_HF'],
                               how='left'
                              ).rename(columns={"HCC_factor":"RAF_Disabled_Disease_Int_DISABLED_HF"})

all_recommendations_I = pd.merge(all_recommendations_I,
                               RAF_Disabled_Disease_Int[RAF_Disabled_Disease_Int['DISABLED_ULCER_']==1][['community_institutional', 'HCC_factor', 'DISABLED_ULCER_']],
                               on=['community_institutional','DISABLED_ULCER_'],
                               how='left'
                              ).rename(columns={"HCC_factor":"RAF_Disabled_Disease_Int_DISABLED_ULCER_"})

all_recommendations_I = pd.merge(all_recommendations_I,
                               RAF_Disabled_Disease_Int[RAF_Disabled_Disease_Int['DISABLED_CANCER']==1][['community_institutional', 'HCC_factor', 'DISABLED_CANCER']],
                               on=['community_institutional', 'DISABLED_CANCER'],
                               how='left'
                              ).rename(columns={"HCC_factor":"RAF_Disabled_Disease_Int_DISABLED_CANCER"})

all_recommendations_I = pd.merge(all_recommendations_I,
                               RAF_Disabled_Disease_Int[RAF_Disabled_Disease_Int['DISABLED_CHR_LUNG']==1][['community_institutional', 'HCC_factor', 'DISABLED_CHR_LUNG']],
                               on=['community_institutional',
                                   'DISABLED_CHR_LUNG'],
                               how='left'
                              ).rename(columns={"HCC_factor":"RAF_Disabled_Disease_Int_DISABLED_CHR_LUNG"})

all_recommendations_I = pd.merge(all_recommendations_I,
                               RAF_Disabled_Disease_Int[RAF_Disabled_Disease_Int['DISABLED_NEURO_']==1][['community_institutional', 'HCC_factor', 'DISABLED_NEURO_']],
                               on=['community_institutional','DISABLED_NEURO_'],
                               how='left'
                              ).rename(columns={"HCC_factor":"RAF_Disabled_Disease_Int_DISABLED_NEURO_"})

all_recommendations_I['RAF_Disabled_Disease_Int'] = (all_recommendations_I['RAF_Disabled_Disease_Int_DISABLED_HF'].fillna(0) + 
                                                    all_recommendations_I['RAF_Disabled_Disease_Int_DISABLED_ULCER_'].fillna(0) + 
                                                    all_recommendations_I['RAF_Disabled_Disease_Int_DISABLED_CANCER'].fillna(0) + 
                                                    all_recommendations_I['RAF_Disabled_Disease_Int_DISABLED_CHR_LUNG'].fillna(0) + 
                                                    all_recommendations_I['RAF_Disabled_Disease_Int_DISABLED_NEURO_'].fillna(0))
                
all_recommendations_I.drop(columns= ['RAF_Disabled_Disease_Int_DISABLED_HF','RAF_Disabled_Disease_Int_DISABLED_ULCER_',
                                     'RAF_Disabled_Disease_Int_DISABLED_CANCER','RAF_Disabled_Disease_Int_DISABLED_CHR_LUNG',
                                     'RAF_Disabled_Disease_Int_DISABLED_NEURO_'],inplace=True)


In [98]:
## finding Risk factor due to HCC counts before recommendation

all_recommendations_lowhcc_count_c = all_recommendations_c[all_recommendations_c['count_HCC_before_recommendations']<5]
all_recommendations_mediumhcc_count_c = all_recommendations_c[(all_recommendations_c['count_HCC_before_recommendations']>=5) &
                                                          (all_recommendations_c['count_HCC_before_recommendations']<10)]
all_recommendations_highhcc_count_c = all_recommendations_c[all_recommendations_c['count_HCC_before_recommendations']>=10]


all_recommendations_lowhcc_count_I = all_recommendations_I[all_recommendations_I['count_HCC_before_recommendations']<5]
all_recommendations_mediumhcc_count_I = all_recommendations_I[(all_recommendations_I['count_HCC_before_recommendations']>=5) &
                                                          (all_recommendations_I['count_HCC_before_recommendations']<10)]
all_recommendations_highhcc_count_I = all_recommendations_I[all_recommendations_I['count_HCC_before_recommendations']>=10]


all_recommendations_lowhcc_count_c["RAF_Payment_HCC_Counts_before"] = 0
all_recommendations_lowhcc_count_I["RAF_Payment_HCC_Counts_before"] = 0

all_recommendations_mediumhcc_count_c = pd.merge(all_recommendations_mediumhcc_count_c,
                                               RAF_Payment_HCC_Counts,
                                               left_on = ['count_HCC_before_recommendations','community_institutional','benefits','Aged_disabled'],
                                               right_on=['Description Label','community_institutional','benefits','Aged_disabled'],
                                               how= 'left').rename(columns={'HCC_factor':'RAF_Payment_HCC_Counts_before'})
all_recommendations_mediumhcc_count_I = pd.merge(all_recommendations_mediumhcc_count_I,
                                               RAF_Payment_HCC_Counts.drop(columns=['benefits','Aged_disabled']),
                                               left_on = ['count_HCC_before_recommendations','community_institutional'],
                                               right_on=['Description Label','community_institutional'],
                                               how= 'left').rename(columns={'HCC_factor':'RAF_Payment_HCC_Counts_before'})

all_recommendations_highhcc_count_c = pd.merge(all_recommendations_highhcc_count_c,
                                               RAF_Payment_HCC_Counts[RAF_Payment_HCC_Counts['Description Label']==10],
                                               left_on = ['community_institutional','benefits','Aged_disabled'],
                                               right_on=['community_institutional','benefits','Aged_disabled'],
                                               how= 'left').rename(columns={'HCC_factor':'RAF_Payment_HCC_Counts_before'})
all_recommendations_highhcc_count_I = pd.merge(all_recommendations_highhcc_count_I,
                                               RAF_Payment_HCC_Counts[RAF_Payment_HCC_Counts['Description Label']==10].drop(columns=['benefits','Aged_disabled']),
                                               left_on = ['community_institutional'],
                                               right_on=['community_institutional'],
                                               how= 'left').rename(columns={'HCC_factor':'RAF_Payment_HCC_Counts_before'})


columns_c = all_recommendations_lowhcc_count_c.columns.to_list()
columns_I = all_recommendations_lowhcc_count_I.columns.to_list()

all_recommendations_c = pd.concat([all_recommendations_lowhcc_count_c[columns_c],
                                 all_recommendations_mediumhcc_count_c[columns_c],
                                 all_recommendations_highhcc_count_c[columns_c]],
                                 axis = 0)
all_recommendations_I = pd.concat([all_recommendations_lowhcc_count_I[columns_I],
                                 all_recommendations_mediumhcc_count_I[columns_I],
                                 all_recommendations_highhcc_count_I[columns_I]],
                                 axis = 0)

In [99]:
## finding Risk factor due to HCC counts before recommendation

all_recommendations_lowhcc_count_c = all_recommendations_c[all_recommendations_c['count_HCC_before_recommendations']<5]
all_recommendations_mediumhcc_count_c = all_recommendations_c[(all_recommendations_c['count_HCC_before_recommendations']>=5) &
                                                          (all_recommendations_c['count_HCC_before_recommendations']<10)]
all_recommendations_highhcc_count_c = all_recommendations_c[all_recommendations_c['count_HCC_before_recommendations']>=10]


all_recommendations_lowhcc_count_I = all_recommendations_I[all_recommendations_I['count_HCC_after_recommendations']<5]
all_recommendations_mediumhcc_count_I = all_recommendations_I[(all_recommendations_I['count_HCC_after_recommendations']>=5) &
                                                          (all_recommendations_I['count_HCC_after_recommendations']<10)]
all_recommendations_highhcc_count_I = all_recommendations_I[all_recommendations_I['count_HCC_after_recommendations']>=10]


all_recommendations_lowhcc_count_c["RAF_Payment_HCC_Counts_after"] = 0
all_recommendations_lowhcc_count_I["RAF_Payment_HCC_Counts_after"] = 0

all_recommendations_mediumhcc_count_c = pd.merge(all_recommendations_mediumhcc_count_c,
                                               RAF_Payment_HCC_Counts,
                                               left_on = ['count_HCC_after_recommendations','community_institutional','benefits','Aged_disabled'],
                                               right_on=['Description Label','community_institutional','benefits','Aged_disabled'],
                                               how= 'left').rename(columns={'HCC_factor':'RAF_Payment_HCC_Counts_after'})
all_recommendations_mediumhcc_count_I = pd.merge(all_recommendations_mediumhcc_count_I,
                                               RAF_Payment_HCC_Counts.drop(columns=['benefits','Aged_disabled']),
                                               left_on = ['count_HCC_after_recommendations','community_institutional'],
                                               right_on=['Description Label','community_institutional'],
                                               how= 'left').rename(columns={'HCC_factor':'RAF_Payment_HCC_Counts_after'})

all_recommendations_highhcc_count_c = pd.merge(all_recommendations_highhcc_count_c,
                                               RAF_Payment_HCC_Counts[RAF_Payment_HCC_Counts['Description Label']==10],
                                               left_on = ['community_institutional','benefits','Aged_disabled'],
                                               right_on=['community_institutional','benefits','Aged_disabled'],
                                               how= 'left').rename(columns={'HCC_factor':'RAF_Payment_HCC_Counts_after'})
all_recommendations_highhcc_count_I = pd.merge(all_recommendations_highhcc_count_I,
                                               RAF_Payment_HCC_Counts[RAF_Payment_HCC_Counts['Description Label']==10].drop(columns=['benefits','Aged_disabled']),
                                               left_on = ['community_institutional'],
                                               right_on=['community_institutional'],
                                               how= 'left').rename(columns={'HCC_factor':'RAF_Payment_HCC_Counts_after'})


columns_c = all_recommendations_lowhcc_count_c.columns.to_list()
columns_I = all_recommendations_lowhcc_count_I.columns.to_list()

all_recommendations_c = pd.concat([all_recommendations_lowhcc_count_c[columns_c],
                                 all_recommendations_mediumhcc_count_c[columns_c],
                                 all_recommendations_highhcc_count_c[columns_c]],
                                 axis = 0)
all_recommendations_I = pd.concat([all_recommendations_lowhcc_count_I[columns_I],
                                 all_recommendations_mediumhcc_count_I[columns_I],
                                 all_recommendations_highhcc_count_I[columns_I]],
                                 axis = 0)

In [100]:
all_recommendations = all_recommendations_c.append(all_recommendations_I, ignore_index = True).sort_values('Target_Beneficiary')

In [101]:
## finding total Risk Adjustment Factor
all_recommendations = all_recommendations.fillna(0)

all_recommendations['RAF_before'] = (all_recommendations['RAF_Age_gender']+
                                     all_recommendations['RAF_Medicaid_OrgnlDsbl']+
                                     all_recommendations['RAF_Disease_Coefficients_before_recommendations']+
                                     all_recommendations['RAF_Disease_intraction']+
                                     all_recommendations['RAF_Disabled_Disease_Int']+
                                     all_recommendations['RAF_Payment_HCC_Counts_before'])

all_recommendations['RAF_after'] = (all_recommendations['RAF_Age_gender']+
                                    all_recommendations['RAF_Medicaid_OrgnlDsbl']+
                                    all_recommendations['RAF_Disease_Coefficients_after_recommendations']+
                                    all_recommendations['RAF_Disease_intraction']+
                                    all_recommendations['RAF_Disabled_Disease_Int']+
                                    all_recommendations['RAF_Payment_HCC_Counts_after'])

all_recommendations = all_recommendations.reset_index(drop=True).sort_values('Target_Beneficiary')

In [102]:
columns = ['Target_Beneficiary', 'Similar_Beneficiary', 'Similarity_score',
           'Target_HCC', 'Simialr_HCC', 'common_HCC', 'individual_recommended_HCC',
           'combined_recommedned_HCC', 'combined_recommedned_HCC_trimmed',
           'HCC_after_recommendations', 'count_HCC_after_recommendations',
           'Gender', 'Age', 'Aged', 'community_institutional', 'Disability_condn',
           'disability', 'benefits', 'medicaid_flag', 'DIABETES_HF', 'HF_CHR_LUNG',
           'HF_KIDNEY', 'CHR_LUNG_CARD_RESP_FAIL','gSubUseDisorder_gPsych_' , 'HF_HCC238', 'DISABLED_HF',
           'DISABLED_ULCER_', 'DISABLED_CANCER','DISABLED_NEURO_', 'DISABLED_CHR_LUNG',
           'Aged_disabled', 'OriginallyDisabled_Medicaid',
           'RAF_Age_gender','RAF_Medicaid_OrgnlDsbl',
           'RAF_Disease_Coefficients_before_recommendations','RAF_Disease_Coefficients_after_recommendations' ,
           'RAF_Disease_intraction', 'RAF_Disabled_Disease_Int',
           'RAF_Payment_HCC_Counts_before','RAF_Payment_HCC_Counts_after' ,'RAF_before','RAF_after']
all_recommendations[columns].head(25)

,Target_Beneficiary,Similar_Beneficiary,Similarity_score,Target_HCC,Simialr_HCC,common_HCC,individual_recommended_HCC,combined_recommedned_HCC,combined_recommedned_HCC_trimmed,HCC_after_recommendations,count_HCC_after_recommendations,Gender,Age,Aged,community_institutional,Disability_condn,disability,benefits,medicaid_flag,DIABETES_HF,HF_CHR_LUNG,HF_KIDNEY,CHR_LUNG_CARD_RESP_FAIL,gSubUseDisorder_gPsych_,HF_HCC238,DISABLED_HF,DISABLED_ULCER_,DISABLED_CANCER,DISABLED_NEURO_,DISABLED_CHR_LUNG,Aged_disabled,OriginallyDisabled_Medicaid,RAF_Age_gender,RAF_Medicaid_OrgnlDsbl,RAF_Disease_Coefficients_before_recommendations,RAF_Disease_Coefficients_after_recommendations,RAF_Disease_intraction,RAF_Disabled_Disease_Int,RAF_Payment_HCC_Counts_before,RAF_Payment_HCC_Counts_after,RAF_before,RAF_after
0,BENE100001,BENE131042,0.877307,"[152, 238]","{226, 228, 37, 326, 201, 238, 213, 22, 152}","{152, 238}","{226, 228, 37, 326, 201, 213, 22}","[326, 226, 228, 37, 201, 213, 22, 229, 38, 18,...","[326, 226]","[152, 238, 326, 226]",4,Female,71.0,1,Community,Non_disabled,1,NonDual,0,0,0,1,0,0,1,0,0,0,0,0,Aged,Medicaid,0.395,0.0,0.783,1.958,0.000,0.0,0.000,0.000,1.178,2.353
1,BENE100001,BENE78294,0.856599,"[152, 238]","{280, 229, 38, 238, 18, 23, 152, 409, 93}","{152, 238}","{229, 38, 18, 23, 280, 409, 93}","[326, 226, 228, 37, 201, 213, 22, 229, 38, 18,...","[326, 226]","[152, 238, 326, 226]",4,Female,71.0,1,Community,Non_disabled,1,NonDual,0,0,0,1,0,0,1,0,0,0,0,0,Aged,Medicaid,0.395,0.0,0.783,1.958,0.000,0.0,0.000,0.000,1.178,2.353
2,BENE100001,BENE102051,0.819628,"[152, 238]","{2, 36, 326, 238, 152, 127}","{152, 238}","{2, 36, 326, 127}","[326, 226, 228, 37, 201, 213, 22, 229, 38, 18,...","[326, 226]","[152, 238, 326, 226]",4,Female,71.0,1,Community,Non_disabled,1,NonDual,0,0,0,1,0,0,1,0,0,0,0,0,Aged,Medicaid,0.395,0.0,0.783,1.958,0.000,0.0,0.000,0.000,1.178,2.353
3,BENE100001,BENE99314,0.816088,"[152, 238]","{152, 153, 238, 63}","{152, 238}","{153, 63}","[326, 226, 228, 37, 201, 213, 22, 229, 38, 18,...","[326, 226]","[152, 238, 326, 226]",4,Female,71.0,1,Community,Non_disabled,1,NonDual,0,0,0,1,0,0,1,0,0,0,0,0,Aged,Medicaid,0.395,0.0,0.783,1.958,0.000,0.0,0.000,0.000,1.178,2.353
4,BENE100001,BENE79348,0.971816,"[152, 238]",{238},{238},{},"[326, 226, 228, 37, 201, 213, 22, 229, 38, 18,...","[326, 226]","[152, 238, 326, 226]",4,Female,71.0,1,Community,Non_disabled,1,NonDual,0,0,0,1,0,0,1,0,0,0,0,0,Aged,Medicaid,0.395,0.0,0.783,1.958,0.000,0.0,0.000,0.000,1.178,2.353
5,BENE100002,BENE22625,0.813009,"[38, 264, 137, 201, 238, 19, 151, 280, 127]","{229, 37, 38, 298, 402, 21, 151, 280}","{280, 38, 151}","{37, 229, 298, 402, 21}","[37, 109, 463, 226, 2, 78, 401, 213, 282, 253,...","[37, 109]","[264, 137, 201, 238, 19, 151, 280, 127, 37, 109]",11,Male,72.0,1,Community,Non_disabled,1,NonDual,0,0,0,0,0,1,0,0,0,0,0,0,Aged,Medicaid,0.396,0.0,4.558,5.702,0.000,0.0,0.444,0.000,5.398,6.098
6,BENE100002,BENE120886,0.826233,"[38, 264, 137, 201, 238, 19, 151, 280, 127]","{280, 109, 238, 463}","{280, 238}","{109, 463}","[37, 109, 463, 226, 2, 78, 401, 213, 282, 253,...","[37, 109]","[264, 137, 201, 238, 19, 151, 280, 127, 37, 109]",11,Male,72.0,1,Community,Non_disabled,1,NonDual,0,0,0,0,0,1,0,0,0,0,0,0,Aged,Medicaid,0.396,0.0,4.558,5.702,0.000,0.0,0.444,0.000,5.398,6.098
7,BENE100002,BENE101706,0.819410,"[38, 264, 137, 201, 238, 19, 151, 280, 127]","{37, 38, 127}","{38, 127}",{37},"[37, 109, 463, 226, 2, 78, 401, 213, 282, 253,...","[37, 109]","[264, 137, 201, 238, 19, 151, 280, 127, 37, 109]",11,Male,72.0,1,Community,Non_disabled,1,NonDual,0,0,0,0,0,1,0,0,0,0,0,0,Aged,Medicaid,0.396,0.0,4.558,5.702,0.000,0.0,0.444,0.000,5.398,6.098
8,BENE100002,BENE127425,0.807181,"[38, 264, 137, 201, 238, 19, 151, 280, 127]","{238, 38, 23}","{38, 238}",{23},"[37, 109, 463, 226, 2, 78, 401, 213, 282, 253,...","[37, 109]","[264, 137, 201, 238, 19, 151, 280, 127, 37, 109]",11,Male,72.0,1,Community,Non_disabled,1,NonDual,0,0,0,0,0,1,0,0,0,0,0,0,Aged,Medicaid,0.396,0.0,4.558,5.702,0.

In [103]:
all_recommendations["RAF_before"].mean()

2.093435951808466

In [104]:
all_recommendations["RAF_after"].mean()

2.852603301673413

In [105]:
(all_recommendations["RAF_after"].mean() - all_recommendations["RAF_before"].mean())/all_recommendations["RAF_before"].mean()*100

36.2641784769733

In [106]:
all_recommendations.to_csv("../data/processed_data/all_recommendations_0.8_top5_withRAF.csv",index=False)

In [116]:
all_recommendations.head(8)

,Target_Beneficiary,Similar_Beneficiary,Similarity_score,Target_HCC,Simialr_HCC,common_HCC,individual_recommended_HCC,combined_recommedned_HCC,combined_recommedned_HCC_trimmed,Gender,Age,Aged,community_institutional,Disability_condn,disability,benefits,medicaid_flag,DIABETES_HF,HF_CHR_LUNG,HF_KIDNEY,CHR_LUNG_CARD_RESP_FAIL,HF_HCC238,DISABLED_HF,DISABLED_ULCER_,DISABLED_CANCER,DISABLED_CHR_LUNG,Aged_disabled,OriginallyDisabled_Medicaid,HCC_after_recommendations,count_HCC_before_recommendations,count_HCC_after_recommendations,gSubUseDisorder_gPsych_,DISABLED_NEURO_,RAF_Age_gender,age_from,age_to,RAF_Medicaid_OrgnlDsbl,RAF_Disease_Coefficients_before_recommendations,RAF_Disease_Coefficients_after_recommendations,RAF_Disease_intraction,RAF_Disabled_Disease_Int,RAF_Payment_HCC_Counts_before,RAF_Payment_HCC_Counts_after,RAF_before,RAF_after
0,BENE100001,BENE131042,0.877307,"[152, 238]","{226, 228, 37, 326, 201, 238, 213, 22, 152}","{152, 238}","{226, 228, 37, 326, 201, 213, 22}","[326, 226, 228, 37, 201, 213, 22, 229, 38, 18,...","[326, 226]",Female,71.0,1,Community,Non_disabled,1,NonDual,0,0,0,1,0,1,0,0,0,0,Aged,Medicaid,"[152, 238, 326, 226]",2,4,0,0,0.395,70,74,0.0,0.783,1.958,0.0,0.0,0.000,0.0,1.178,2.353
1,BENE100001,BENE78294,0.856599,"[152, 238]","{280, 229, 38, 238, 18, 23, 152, 409, 93}","{152, 238}","{229, 38, 18, 23, 280, 409, 93}","[326, 226, 228, 37, 201, 213, 22, 229, 38, 18,...","[326, 226]",Female,71.0,1,Community,Non_disabled,1,NonDual,0,0,0,1,0,1,0,0,0,0,Aged,Medicaid,"[152, 238, 326, 226]",2,4,0,0,0.395,70,74,0.0,0.783,1.958,0.0,0.0,0.000,0.0,1.178,2.353
2,BENE100001,BENE102051,0.819628,"[152, 238]","{2, 36, 326, 238, 152, 127}","{152, 238}","{2, 36, 326, 127}","[326, 226, 228, 37, 201, 213, 22, 229, 38, 18,...","[326, 226]",Female,71.0,1,Community,Non_disabled,1,NonDual,0,0,0,1,0,1,0,0,0,0,Aged,Medicaid,"[152, 238, 326, 226]",2,4,0,0,0.395,70,74,0.0,0.783,1.958,0.0,0.0,0.000,0.0,1.178,2.353
3,BENE100001,BENE99314,0.816088,"[152, 238]","{152, 153, 238, 63}","{152, 238}","{153, 63}","[326, 226, 228, 37, 201, 213, 22, 229, 38, 18,...","[326, 226]",Female,71.0,1,Community,Non_disabled,1,NonDual,0,0,0,1,0,1,0,0,0,0,Aged,Medicaid,"[152, 238, 326, 226]",2,4,0,0,0.395,70,74,0.0,0.783,1.958,0.0,0.0,0.000,0.0,1.178,2.353
4,BENE100001,BENE79348,0.971816,"[152, 238]",{238},{238},{},"[326, 226, 228, 37, 201, 213, 22, 229, 38, 18,...","[326, 226]",Female,71.0,1,Community,Non_disabled,1,NonDual,0,0,0,1,0,1,0,0,0,0,Aged,Medicaid,"[152, 238, 326, 226]",2,4,0,0,0.395,70,74,0.0,0.783,1.958,0.0,0.0,0.000,0.0,1.178,2.353
5,BENE100002,BENE22625,0.813009,"[38, 264, 137, 201, 238, 19, 151, 280, 127]","{229, 37, 38, 298, 402, 21, 151, 280}","{280, 38, 151}","{37, 229, 298, 402, 21}","[37, 109, 463, 226, 2, 78, 401, 213, 282, 253,...","[37, 109]",Male,72.0,1,Community,Non_disabled,1,NonDual,0,0,0,0,0,0,0,0,0,0,Aged,Medicaid,"[264, 137, 201, 238, 19, 151, 280, 127, 37, 109]",9,11,1,0,0.396,70,74,0.0,4.558,5.702,0.0,0.0,0.444,0.0,5.398,6.098
6,BENE100002,BENE120886,0.826233,"[38, 264, 137, 201, 238, 19, 151, 280, 127]","{280, 109, 238, 463}","{280, 238}","{109, 463}","[37, 109, 463, 226, 2, 78, 401, 213, 282, 253,...","[37, 109]",Male,72.0,1,Community,Non_disabled,1,NonDual,0,0,0,0,0,0,0,0,0,0,Aged,Medicaid,"[264, 137, 201, 238, 19, 151, 280, 127, 37, 109]",9,11,1,0,0.396,70,74,0.0,4.558,5.702,0.0,0.0,0.444,0.0,5.398,6.098
7,BENE100002,BENE101706,0.819410,"[38, 264, 137, 201, 238, 19, 151, 280, 127]","{37, 38, 127}","{38, 127}",{37},"[37, 109, 463, 226, 2, 78, 401, 213, 282, 253,...","[37, 109]",Male,72.0,1,Community,Non_disabled,1,NonDual,0,0,0,0,0,0,0,0,0,0,Aged,Medicaid,"[264, 137, 201, 238, 19, 151, 280, 127, 37, 109]",9,11,1,0,0.396,70,74,0.0,4.558,5.702,0.0,0.0,0.444,0.0,5.398,6.098


In [113]:
all_recommendations_exploded_c.head()

,Target_Beneficiary,HCC_after_recommendations,community_institutional,benefits,Aged_disabled,HCC_code,Benefits,RAF_Disease_Coefficients_after_recommendations
0,BENE100001,152,Community,NonDual,Aged,152,NonDual,0.484
1,BENE100001,238,Community,NonDual,Aged,238,NonDual,0.299
2,BENE100001,326,Community,NonDual,Aged,326,NonDual,0.815
3,BENE100001,226,Community,NonDual,Aged,226,NonDual,0.360
4,BENE100002,264,Community,NonDual,Aged,264,NonDual,0.455


In [108]:
all_recommendations_exploded_c[all_recommendations_exploded_c['Target_Beneficiary']=='BENE100002'][['Target_Beneficiary','HCC_after_recommendations','RAF_Disease_Coefficients_after_recommendations']]

,Target_Beneficiary,HCC_after_recommendations,RAF_Disease_Coefficients_after_recommendations
4,BENE100002,264,0.455
5,BENE100002,137,0.424
6,BENE100002,201,0.245
7,BENE100002,238,0.299
8,BENE100002,19,1.798
9,BENE100002,151,0.511
10,BENE100002,280,0.319
11,BENE100002,127,0.341
12,BENE100002,37,0.166
13,BENE100002,109,1.144


In [109]:
all_recommendations_exploded_c[all_recommendations_exploded_c['Target_Beneficiary']=='BENE100002']['RAF_Disease_Coefficients_after_recommendations'].sum()

5.702000000000001

In [110]:
all_recommendations[all_recommendations['Target_Beneficiary']=='BENE100002'][['Target_Beneficiary','RAF_Age_gender','RAF_Disease_Coefficients_after_recommendations','RAF_Disease_intraction','RAF_Disabled_Disease_Int','RAF_Payment_HCC_Counts_after','RAF_after']]

,Target_Beneficiary,RAF_Age_gender,RAF_Disease_Coefficients_after_recommendations,RAF_Disease_intraction,RAF_Disabled_Disease_Int,RAF_Payment_HCC_Counts_after,RAF_after
5,BENE100002,0.396,5.702,0.0,0.0,0.0,6.098
6,BENE100002,0.396,5.702,0.0,0.0,0.0,6.098
7,BENE100002,0.396,5.702,0.0,0.0,0.0,6.098
8,BENE100002,0.396,5.702,0.0,0.0,0.0,6.098
9,BENE100002,0.396,5.702,0.0,0.0,0.0,6.098


In [111]:
all_recommendations[all_recommendations['Target_Beneficiary']=='BENE100002'][['Target_Beneficiary','RAF_Age_gender',
                                                                              'RAF_Disease_Coefficients_before_recommendations','RAF_Disease_Coefficients_after_recommendations','RAF_Disease_intraction',
                                                                              'RAF_Disabled_Disease_Int','RAF_Payment_HCC_Counts_before','RAF_Payment_HCC_Counts_after','RAF_before','RAF_after']]

,Target_Beneficiary,RAF_Age_gender,RAF_Disease_Coefficients_before_recommendations,RAF_Disease_Coefficients_after_recommendations,RAF_Disease_intraction,RAF_Disabled_Disease_Int,RAF_Payment_HCC_Counts_before,RAF_Payment_HCC_Counts_after,RAF_before,RAF_after
5,BENE100002,0.396,4.558,5.702,0.0,0.0,0.444,0.0,5.398,6.098
6,BENE100002,0.396,4.558,5.702,0.0,0.0,0.444,0.0,5.398,6.098
7,BENE100002,0.396,4.558,5.702,0.0,0.0,0.444,0.0,5.398,6.098
8,BENE100002,0.396,4.558,5.702,0.0,0.0,0.444,0.0,5.398,6.098
9,BENE100002,0.396,4.558,5.702,0.0,0.0,0.444,0.0,5.398,6.098
